# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load csv into dataframe
cities_df = pd.read_csv('output_data/cities.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure g_key
gmaps.configure(api_key=g_key)

In [4]:
#Create Heatmap
locations = cities_df[["Lat", "Lon"]]
humidity = cities_df["Humidity"]

figure = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 250, point_radius = 4)
figure.add_layer(heatmap)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down cities to desired criteria
narrowed_cities_df = cities_df.loc[(cities_df["Max Temp"] <80) & (cities_df["Max Temp"]  >70) 
                                   & (cities_df["Windspeed"] < 10) & (cities_df["Cloudiness"] == 0)]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Create dataframe with hotel column
hotel_df = narrowed_cities_df[["City", "Country", "Lat", "Lon"]]
hotel_df["Hotel Name"] = ""

/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
#Design and execute api search for hotels
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

for index, row in hotel_df.iterrows():
    lat= row["Lat"]
    lng= row["Lon"]
    
    query_url = (f"{url}key={g_key}&location={lat},{lng}&radius=5000&type=lodging")

    try:
        time.sleep(1)
        response = requests.get(query_url).json()
        
        hotel_name = response["results"][0]["name"]
        print(hotel_name)
        
        #Fill dataframe with responses
        hotel_df.at[index,"Hotel Name"] = hotel_name
         
    except Exception as e:

        hotel_df.at[index,"Hotel Name"] = "no hotel found"
        print(e)


Mishan Hotel
昆明佳华广场酒店
Marè Resort
Gondolas Forum grandchildren and grandparents
Hotel San Domenico
list index out of range
Gostinitsa Aray
安顺麦吉风格酒店
City Hotel
Casa Das Faias
Aracati Hotel
Bahiadomizil
El Medina


In [8]:
#Display dataframe of cities and their nearby hotel
hotel_df

,City,Country,Lat,Lon,Hotel Name
61,mishan,CN,45.55,131.88,Mishan Hotel
114,kunming,CN,25.04,102.72,昆明佳华广场酒店
175,trani,IT,41.27,16.42,Marè Resort
271,salamiyah,SY,35.01,37.05,Gondolas Forum grandchildren and grandparents
291,soverato,IT,38.69,16.55,Hotel San Domenico
343,mayor pablo lagerenza,PY,-19.93,-60.77,no hotel found
413,zaysan,KZ,47.47,84.87,Gostinitsa Aray
428,anshun,CN,26.25,105.93,安顺麦吉风格酒店
506,surt,LY,31.21,16.59,City Hotel
525,lagoa,PT,39.05,-27.98,Casa Das Faias


In [9]:
#Remove Cities for which there are no nearby hotels
hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!= "no hotel found"]
hotel_df

,City,Country,Lat,Lon,Hotel Name
61,mishan,CN,45.55,131.88,Mishan Hotel
114,kunming,CN,25.04,102.72,昆明佳华广场酒店
175,trani,IT,41.27,16.42,Marè Resort
271,salamiyah,SY,35.01,37.05,Gondolas Forum grandchildren and grandparents
291,soverato,IT,38.69,16.55,Hotel San Domenico
413,zaysan,KZ,47.47,84.87,Gostinitsa Aray
428,anshun,CN,26.25,105.93,安顺麦吉风格酒店
506,surt,LY,31.21,16.59,City Hotel
525,lagoa,PT,39.05,-27.98,Casa Das Faias
526,tefe,BR,-3.35,-64.71,Aracati Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lon"]]

In [13]:
# Add marker layer ontop of heat map

fig = gmaps.figure()
hotel_layer = gmaps.marker_layer(locations2,
                                 info_box_content=hotel_info)
fig.add_layer(heatmap)
fig.add_layer(hotel_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))